In [8]:
import pickle 
import requests
import datetime as dt
import pandas as pd
import scipy.sparse as sp
import numpy as np
from utils.csv_to_vec import csv_to_vec
from data_collection.weather_forecast import get_12hr_forecast
from utils.prepare_report import transform 
from bs4 import BeautifulSoup
import re

In [2]:
pac = pickle.load(open('models/2__nearest_centroid__v1.pkl', 'rb'))
tfidf_vec = pickle.load(open('data/3_isw_vectorized_data/tfidf_transformer.pkl', 'rb'))
cv = pickle.load(open('data/3_isw_vectorized_data/cv.pkl', 'rb'))

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Loading weather data

In [4]:
df_weather = pd.DataFrame(get_12hr_forecast('Kyiv'))
df_weather

datetime  datetimeEpoch  temp  feelslike  humidity   dew  precip   
0   22:00:00     1683140400  15.7       15.7     58.55   7.6     0.0  \
1   23:00:00     1683144000  14.0       14.0     78.39  10.3     0.0   
2   00:00:00     1683147600  13.0       13.0     79.83   9.6     0.0   
3   01:00:00     1683151200  12.1       12.1     80.25   8.8     0.0   
4   02:00:00     1683154800  11.4       11.4     80.69   8.2     0.0   
5   03:00:00     1683158400  10.7       10.7     83.39   8.0     0.0   
6   04:00:00     1683162000  10.2       10.2     85.05   7.8     0.0   
7   05:00:00     1683165600   9.8        9.8     86.77   7.7     0.0   
8   06:00:00     1683169200   9.6        9.6     87.34   7.6     0.0   
9   07:00:00     1683172800  10.0       10.0     86.20   7.8     0.0   
10  08:00:00     1683176400  11.2       11.2     80.12   7.9     0.0   
11  09:00:00     1683180000  12.3       12.3     76.04   8.2     0.0   

    precipprob  snow  snowdepth preciptype  windgust  windspeed  winddir   
0          0.0   0.0        0.0       None      14.4        6.5    346.0  \
1          0.0   0.0        0.0       None      24.8        9.7     11.0   
2          0.0   0.0        0.0       None      23.8        7.9     16.7   
3          0.0   0.0        0.0       None      19.8        6.8     13.8   
4          0.0   0.0        0.0       None      16.9        5.8     18.3   
5          0.0   0.0        0.0       None      13.7        4.7     16.0   
6          0.0   0.0        0.0       None      11.9        5.0     29.0   
7          0.0   0.0        0.0       None      12.2        4.0     29.9   
8          0.0   0.0        0.0       None       9.4        3.2     30.3   
9          0.0   0.0        0.0       None      10.1        4.0     38.5   
10         0.0   0.0        0.0       None      14.8        5.0     43.7   
11         0.0   0.0        0.0       None      16.2        5.0     65.0   

    pressure  visibility  cloudcover  solarradiation  solarenergy  uvindex   
0     1022.0        24.1        93.8             0.0          NaN      0.0  \
1     1022.0        24.1       100.0             0.0          NaN      0.0   
2     1023.0        24.1        92.4             0.0          NaN      0.0   
3     1023.0        24.1        82.8             0.0          NaN      0.0   
4     1023.0        24.1       100.0             0.0          NaN      0.0   
5     1023.0        24.1       100.0             0.0          NaN      0.0   
6     1023.0        24.1       100.0             0.0          NaN      0.0   
7     1023.0        24.1       100.0             0.0          NaN      0.0   
8     1023.0        24.1       100.0             4.0          0.0      0.0   
9     1023.0        24.1        98.5            49.0          0.2      0.0   
10    1023.0        24.1        99.1           152.0          0.5      2.0   
11    1023.0        24.1       100.0           268.0          1.0      3.0   

    severerisk        conditions                 icon stations source  
0         10.0          Overcast               cloudy     None   fcst  
1         10.0          Overcast               cloudy     None   fcst  
2         10.0          Overcast               cloudy     None   fcst  
3         10.0  Partially cloudy  partly-cloudy-night     None   fcst  
4         10.0          Overcast               cloudy     None   fcst  
5         10.0          Overcast               cloudy     None   fcst  
6         10.0          Overcast               cloudy     None   fcst  
7         10.0          Overcast               cloudy     None   fcst  
8         10.0          Overcast               cloudy     None   fcst  
9         10.0          Overcast               cloudy     None   fcst  
10        10.0          Overcast               cloudy     None   fcst  
11        10.0          Overcast               cloudy     None   fcst

In [5]:
exclude = {
    'datetime',
    'datetimeEpoch',
    'feelslike',
    'preciptype',
    'conditions',
    'solarenergy',
    'icon',
    'stations',
    'source'
}

In [ ]:
time = df_weather['datetime']
df_weather = df_weather.drop(columns=exclude)
df_weather = df_weather.fillna(method='ffill')

In [10]:
df_reg = pd.read_csv('data/regions.csv')
df_weather['region_id'] = int(df_reg[df_reg['center_city_en']=='Kyiv']['region_id'].values)
df_weather

temp  humidity   dew  precip  precipprob  snow  snowdepth  windgust   
0   15.7     58.55   7.6     0.0         0.0   0.0        0.0      14.4  \
1   14.0     78.39  10.3     0.0         0.0   0.0        0.0      24.8   
2   13.0     79.83   9.6     0.0         0.0   0.0        0.0      23.8   
3   12.1     80.25   8.8     0.0         0.0   0.0        0.0      19.8   
4   11.4     80.69   8.2     0.0         0.0   0.0        0.0      16.9   
5   10.7     83.39   8.0     0.0         0.0   0.0        0.0      13.7   
6   10.2     85.05   7.8     0.0         0.0   0.0        0.0      11.9   
7    9.8     86.77   7.7     0.0         0.0   0.0        0.0      12.2   
8    9.6     87.34   7.6     0.0         0.0   0.0        0.0       9.4   
9   10.0     86.20   7.8     0.0         0.0   0.0        0.0      10.1   
10  11.2     80.12   7.9     0.0         0.0   0.0        0.0      14.8   
11  12.3     76.04   8.2     0.0         0.0   0.0        0.0      16.2   

    windspeed  winddir  pressure  visibility  cloudcover  solarradiation   
0         6.5    346.0    1022.0        24.1        93.8             0.0  \
1         9.7     11.0    1022.0        24.1       100.0             0.0   
2         7.9     16.7    1023.0        24.1        92.4             0.0   
3         6.8     13.8    1023.0        24.1        82.8             0.0   
4         5.8     18.3    1023.0        24.1       100.0             0.0   
5         4.7     16.0    1023.0        24.1       100.0             0.0   
6         5.0     29.0    1023.0        24.1       100.0             0.0   
7         4.0     29.9    1023.0        24.1       100.0             0.0   
8         3.2     30.3    1023.0        24.1       100.0             4.0   
9         4.0     38.5    1023.0        24.1        98.5            49.0   
10        5.0     43.7    1023.0        24.1        99.1           152.0   
11        5.0     65.0    1023.0        24.1       100.0           268.0   

    uvindex  severerisk  region_id  
0       0.0        10.0         10  
1       0.0        10.0         10  
2       0.0        10.0         10  
3       0.0        10.0         10  
4       0.0        10.0         10  
5       0.0        10.0         10  
6       0.0        10.0         10  
7       0.0        10.0         10  
8       0.0        10.0         10  
9       0.0        10.0         10  
10      2.0        10.0         10  
11      3.0        10.0         10

In [11]:
df_weather = sp.csr_matrix(df_weather.values.astype(float))

In [12]:
report_date = dt.datetime.now() - dt.timedelta(days=1)
report_date = report_date.strftime('%B-%e-%Y').replace(' ', '').lower()
report_date

'may-2-2023'

In [13]:
base_url = 'https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment'
isw_data = requests.get(f'{base_url}-{report_date}').text
isw_data

'<!DOCTYPE html>\n<html xmlns:og="http://opengraphprotocol.org/schema/" lang="en" dir="ltr">\n\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<meta name="Generator" content="Drupal 7 (http://drupal.org)" />\n<link rel="canonical" href="/backgrounder/russian-offensive-campaign-assessment-may-2-2023" />\n<link rel="shortlink" href="/node/5213" />\n<link rel="shortcut icon" href="https://www.understandingwar.org/sites/default/themes/isw/isw.ico" type="image/vnd.microsoft.icon" />\n<meta name="keywords" content="Ukraine Project,Institute for the Study of War, ISW, War, Institute, Study, ORBAT, Report, Iraq, Iran, Afghanistan, Middle East, Libya, Security,,Institute for the Study of War, ISW, War, Institute, Study, ORBAT, Report, Iraq, Iran, Afghanistan, Middle East, Libya, Security," />\n<meta name="description" content="Russian Defense Minister Sergei Shoigu claimed that the Russian defense industrial base (DIB) is increasing its production of precision 

In [14]:
soup = BeautifulSoup(isw_data, 'html.parser')
report = transform(soup)
report

'riley bailey grace mappes kateryna stepanenko mason clark two thousand and twenty three 7 00 pm etrussian defense minister sergei shoigu claimed russian defense industrial base dib increasing production precision missile use ukraine shoigu highlighted state owned tactical missile corporation model defense enterprise stating successfully beginning mass production missile develop plan double current production output near term shoigu focus precision missile production aligns shift russian ministry defense mod rhetoric focusing russia use precision missile strike military infrastructure target ukrainian rear area likely aiming similarly appear proactive demonstrative positive russian action amid growing concern russian information space potential ukrainian counteroffensive shoigu likely seek deflect intensifying accusation russian ministry defense mod not provide sufficient ammunition russian force shoigu stated russian defense industrial base dib increased production pace output volume 

In [15]:
tfidf = csv_to_vec(report, cv, tfidf_vec)

In [16]:
tfidf

{'resolution': 0.171,
 'serviceman': 0.135,
 'bakhmut': 0.133,
 'production': 0.132,
 'avdiivka': 0.125,
 'counteroffensive': 0.123,
 'defense': 0.122,
 'mitigate': 0.122,
 'shoigu': 0.118,
 'aggressor': 0.117,
 'celebration': 0.114,
 'area': 0.112,
 'dib': 0.107,
 'china': 0.107,
 'strike': 0.101,
 'milblogger': 0.101,
 'kreminna': 0.101,
 'shock': 0.1,
 'conducted': 0.098,
 'mod': 0.096,
 'state': 0.095,
 'victory': 0.092,
 'attack': 0.091,
 'un': 0.091,
 'casualty': 0.088,
 'belarusian': 0.087,
 'relative': 0.084,
 'two': 0.083,
 'wagner': 0.082,
 'kremlin': 0.082,
 'near': 0.081,
 'prigozhin': 0.08,
 'instructing': 0.078,
 'cancellation': 0.078,
 'upcoming': 0.077,
 'incentivize': 0.075,
 'ihor': 0.074,
 'thousand': 0.072,
 'died': 0.071,
 'space': 0.071,
 'voted': 0.071,
 'insurance': 0.071,
 'frame': 0.071,
 'accusation': 0.07,
 'precision': 0.07,
 'security': 0.069,
 'veteran': 0.068,
 'remainder': 0.067,
 'output': 0.067,
 'spokesperson': 0.066,
 'informational': 0.065,
 'meduz

In [17]:
cv_vector = cv.transform([report])
tfidf_vector = tfidf_vec.transform(cv_vector) 

In [18]:
pd.DataFrame(tfidf_vector.toarray())

0         1     2     3     4         5     6     7     8     9      
0  0.024043  0.031772   0.0   0.0   0.0  0.021528   0.0   0.0   0.0   0.0  \

   10    11    12    13    14    15    16    17        18    19    20    21     
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.039071   0.0   0.0   0.0  \

   22        23    24    25    26    27    28    29    30    31    32    33     
0   0.0  0.046196   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   34    35    36    37    38    39    40    41    42    43    44    45     
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   46    47    48    49    50    51        52    53    54    55    56    57     
0   0.0   0.0   0.0   0.0   0.0   0.0  0.020399   0.0   0.0   0.0   0.0   0.0  \

   58    59    60    61    62        63    64    65    66        67    68     
0   0.0   0.0   0.0   0.0   0.0  0.030734   0.0   0.0   0.0  0.038354   0.0  \

   69    70        71    72    73        74    75    76    77    78    79     
0   0.0   0.0  0.020252   0.0   0.0  0.040504   0.0   0.0   0.0   0.0   0.0  \

   80    81    82    83    84    85    86    87    88    89    90    91     
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   92        93    94    95    96    97    98    99    100   101   102   103    
0   0.0  0.024043   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   104   105   106   107   108   109   110   111   112   113   114   115    
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   116   117   118   119   120   121   122   123   124   125   126   127    
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   128   129   130   131   132   133   134   135   136   137   138   139    
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   140   141   142   143   144       145   146   147   148   149   150   151    
0   0.0   0.0   0.0   0.0   0.0  0.023794   0.0   0.0   0.0   0.0   0.0   0.0  \

   152   153   154   155   156   157   158   159   160   161   162   163    
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   164   165   166   167   168   169   170   171   172   173   174   175    
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   176   177   178   179   180   181   182   183   184   185   186   187    
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   188   189   190   191   192   193   194   195   196   197   198   199    
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   200   201   202   203   204   205   206   207   208   209   210   211    
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   212   213   214   215   216   217   218   219   220   221   222   223    
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   224   225   226   227   228   229   230   231   232   233   234   235    
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   236   237   238   239   240   241   242   243   244   245   246   247    
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   248   249   250   251       252   253   254   255   256   257   258   259    
0   0.0   0.0   0.0   0.0  0.049087   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   260   261   262   263   264   265   266   267   268   269   270   271    
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   272   273       274   275   276   277   278   279   280   281   282   283    
0   0.0   0.0  0.036496   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  \

   284   285   286   287   288   289   290       291   292   293       294    
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.020703   0.0   0.0  0.027622  \

   295   296   297   298   299   300   301   302   303   304    

In [19]:
tfidf_mat = tfidf_vector
for i in range(11):
    tfidf_mat = sp.vstack((tfidf_mat, tfidf_vector), format='csr')

tfidf_mat.shape

(12, 9343)

In [20]:
X = sp.hstack((df_weather, tfidf_mat), format='csr')

In [21]:
prediction = pac.predict(X)
prediction = list(zip(time.values, prediction))
print(prediction)

[('22:00:00', False), ('23:00:00', False), ('00:00:00', False), ('01:00:00', False), ('02:00:00', False), ('03:00:00', False), ('04:00:00', False), ('05:00:00', False), ('06:00:00', False), ('07:00:00', True), ('08:00:00', True), ('09:00:00', True)]
